# Positional encoding

En el lenguaje el orden de las palabras es muy importante en las frases, ya que `Juan quiere a María` no significa lo mismo que `María quiere a Juan`, en las dos frases hay una persona que quiere a otra, pero por cambiar solo dos palabras de sitio en cada frase es distinta la persona que quiere a la otra. O en las frases `estoy seguro de que no debemos hacerlo` y `no estoy seguro de debamos hacerlo` la posición de la palabra `no` cambia bastante el significado de la frase. Por lo que es importante tener algún mecanismo para poder determinar el orden de cada palabra en la frase

Antes de los transformers, para problemas de lenguaje se usaban modelos recurrentes, es decir, modelos a los que les entraba una palabra y generaban una salida, a continuación les entraba otra palabra y generaban una segunda salida. De esta manera no hacía falta saber la posición de las palabras, porque entraban al modelo una a una. Mientras que a los transformer les entra la frase entera y mediante los mecanismos de atención se hace la magia

Estos modelos recurrentes tenían el problema de que si la frase era muy larga, cuando iban por las palabras finales se habían olvidado de las primeras palabras, por lo que los resultados que se obtenían no eran muy buenos

Con los transformers, al introducir la frase entera al modelo este problema se soluciona, pero se añade el problema de no saber el orden de las palabras, por eso se tuvo que meter el mecanismo de `positional encoding`

Una primera intuición de cómo solucionar esto sería poner índices a las palabras en función de su orden, es decir, la primera palabra tendría el índice 0, la segunda el índice 1, y así sucesivamente. Sin embargo esto tiene el problema de que para que el modelo sea derivable y así poder hacer la operación del descenso del gradiente no podemos poner el índice a secas y con un `if` o un `for` ir buscando palabras. Por lo que ese índice se tendría que sumar, multiplicar o cualquier otra operación al vector de embedding de la palabra. Por lo que a medida que avanzamos en la frase, el índice será mayor, lo que supondrá números mayores y que con las multiplicaciones de los pesos puede hacer que se le de más importancia a las últimas palabras de la frase que a las primeras

Otro efecto negativo de esto es que como hemos explicado, los embedding corresponden a la posición de las palabras en un espacio vectorial, en el que las palabras que tienen un significado parecido suelen estar colocadas juntas. Por lo que si se modifican de esta manera, algunas pueden llegar a modificarse tanto que se muevan a otra zona de otras palabras con otro significado. Por lo que todo lo que ganamos con el word embedding lo perdemos con el positional encoding

Por tanto necesitamos algo que nos permita saber la posición y además al añadírselo al vector de embedding no altere el resultado de la inferencia o del entrenamiento de la red

Que es lo que se hace en la capa de positional encoding del transformer

![Encoder positional encoding](Imagenes/transformer_architecture_model_encoder_positional_embedding.png)

## Pocicion mediante seno y coseno

La solución que proponen en el paper es determinar la posición mediante un seno y un coseno

![positional encoding](Imagenes/positional_encoding.png)

Como sabemos, el valor del seno y del coseno va de entre -1 y 1, por lo que al añadirlos al vector de embedding solo va a modificar cada valor del vector entre -1 y 1. Por tanto, las palabras no se van a ver muy desplazadas dentro del espacio vectorial. Cada valor del vector de una palabra cambiará entre -1 y 1, lo que hará que se mieva la palabra dentro de ese espacio vectorial, pero el desplazamiento será tan pequeño que seguirá dentro de la zona de palabras con el mismo significado

En la siguiente figura podemos ver como la palabra `batery` está en una zona del espacio vectorial y se ve media circunferencia de puntos de las zonas a las que podría desplazarse

[![word embedding displacement](Imagenes/positional_encoding_embeding_displacement.png)](https://e2eml.school/transformers.html)

En realidad en el ejemplo de la imagen, si imaginamos un círculo alrededor de la imagen, todo ese área es la zona a la que se podría desplazar la palabra tras el positional encoding

En el siguiente espacio vectorial cada palabras se movería en un espacio correspondiente a una esfera de radio 1 alrededor de ella. Podemos ver que los ejes van desde -1000 hasta 1000, más o menos, en cada eje. Por lo que un desplazamiento de 1 en cualquiera de cada uno de los ejes no significaría apenas un desplazamiento 

![word embedding 3 dimmensions](Imagenes/word_embedding_3_dimmension.png)

Volvamos a la fórmula

![positional encoding](Imagenes/positional_encoding.png)

Lo que se está diciendo con estas fórmulas es que se va a crear una matriz de positional encoding que va a tener tantas filas como la frase más larga que pueda entrar al transformer y tantas columnas como las dimensiones de nuestro word embedding

Esto lo explicaremos más adelante, pero para entrenar a los transformers no les puedes introducir primero una frase de 10 palabras, luego otra de 20 palabras, etc. Porque eso corresponde a matrices.
Como hemos visto en los entrenamientos de redes neuronales se hacen batches de entradas y salidas, se meten a la red y se hace el descenso del gradiente. Por lo que si una frase tiene 10 palabras, otra 20 palabras y así, no se pueden hacer batches. Así que lo que se hace es definir una longitud máxima de frase, de manera que las frases que tengan una longitud menor se rellenan con embeddings que no corresponden a nada.
Por esto la matriz de positional encoding tendrá tantas filas como se defina esta longitud

Por otro lado hemos dicho que nuestro word embedding es un espacio vectorial de tantas dimensiones que queramos. Esto quiere decir que para cada palabra se creará un vector con tantas posiciones o de una longitud como la dimensión que definamos para el word embedding. Si defnimos un word embedding de 512 eso quiere decir que cada palabra corresponderá a un vector de tamaño 512.
Por eso, el número de columnas de nuestro positional encoding será igual a la dimensión de nuestro word embedding

Teniendo claro qué corresponde a cada dimensión de nuestro positional encoding, lo que nos dice la fórmula, es que para cada posición (para cada fila) las posiciones pares (0, 2, 4, ...) tendrán un valor determinado por un seno, y que cada posición impar (1, 3, 5, ...) tendrá un valor determinado con un coseno

El resultado del seno o del coseno depende de la posición de la palabra en la frase, de la posición del vector de embedding y del tamaño del word embedding

Por lo que podemos ver, se van a crear vectores con valores entre -1 y 1 de manera que su valor dependerá de la posición de la palabra en la frase

Una vez hemos podido crear la matriz de positional encoding, lo que haremos será sumarla a la matriz de word embedding, sabiendo que esto mdesplazará cada palabra dentro del espacio vectorial, pero será un desplazamiento muy pequeño y no alterará prácticamente su significado semántico

¿Por qué los investigadores elgieron una combinación de seno y coseno, en vez de solo senos o solo cosenos? Supongo que probarían varias opciones y se quedaron con esta que es la que mejor les funcionó

## Implementación del positional encoding

Vamos a crear directamente la clase de positional encoding que hace todo lo que hemos explicado y alguna cosa más que ahora explicaremos

In [3]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, max_sequence_len, embedding_model_dim):
        """
        Args:
            seq_len: length of input sequence
            embed_model_dim: demension of embedding
        """
        super().__init__()
        self.embedding_dim = embedding_model_dim

        # create constant 'positional_encoding' matrix with values dependant on pos and i
        positional_encoding = torch.zeros(max_sequence_len, self.embedding_dim)
        for pos in range(max_sequence_len):
            for i in range(0, self.embedding_dim, 2):
                positional_encoding[pos, i]     = math.sin(pos / (10000 ** ((2 *     i) / self.embedding_dim)))
                positional_encoding[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i+1)) / self.embedding_dim)))
        positional_encoding = positional_encoding.unsqueeze(0)
        self.register_buffer('positional_encoding', positional_encoding)
        # Esta última línea es equivalente a hacer self.positional_encoding = positional_encoding
        # sin embargo al hacerlo así PyTorch no lo considerará parte del estado del modelo.
        # Hay varias implicaciones de esto:
        #  * Movimiento de dispositivos: Cuando se mueve el modelo a la GPU con model.to(device), 
        #    PyTorch automáticamente moverá todos los parámetros y buffers registrados al dispositivo especificado.
        #    Sin embargo, no moverá los tensores que no sean parámetros o buffers registrados. Por lo tanto, 
        #    si se asigna positional_encoding a self.positional_encoding directamente, habría que moverlo 
        #    manualmente a la GPU.
        #  * Serialización: Cuando se guarda el modelo con torch.save(model.state_dict(), PATH), PyTorch guardará 
        #    todos los parámetros y buffers registrados del modelo. Pero no guardará tensores que no son parámetros 
        #    o buffers registrados. Por lo tanto, si se asigna positional_encoding a self.positional_encoding 
        #    directamente, no se guardará cuando se guarde el estado del modelo.
        #  * Modo de evaluación: Algunos métodos, como model.eval(), cambian el comportamiento de ciertas capas del 
        #    modelo (como Dropout o BatchNorm) dependiendo de si el modelo está en modo de entrenamiento o evaluación.
        #    Para que estas capas funcionen correctamente, PyTorch necesita conocer su estado actual, que se almacena
        #    en sus parámetros y buffers registrados. Si positional_encoding no está registrado como un buffer, 
        #    entonces no será afectado por el cambio de modo.
        # En resumen, si no se usa register_buffer para positional_encoding, se tendrían que manejar estas cosas 
        # manualmente, lo cual podría ser propenso a errores.
        # La principal ventaja de usar register_buffer() es que PyTorch se encargará de estas cosas por nosotros.


    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            x: output
        """
        # make embeddings relatively larger
        x = x * math.sqrt(self.embedding_dim)
        
        # add encoding matrix to embedding (x)
        sequence_len = x.size(1)
        # x = x + torch.autograd.Variable(self.positional_encoding[:,:sequence_len], requires_grad=False)
        x = x + self.positional_encoding[:,:sequence_len]
        return x

Viendo la clase se puede ver que al inicializar la clase se crea la matriz de positional encoding con la fórmula que propone el paper. Por último como se explica en los comentarios del código se hace `self.register_buffer('positional_encoding', positional_encoding)` y no `self.positional_encoding = positional_encoding` para que Pytorch tenga en cuenta esa matriz

Viendo el método `forward` vemos que lo que primero se hace es multiplicar la matriz de embedding por la raiz cuadrada de la dimensión de nuestro word embedding, esto se hace porque si los valores de la matriz de embedding son pequeños, la perturbación de la amtriz de positional encoding va a ser significativa. De modo que se hace a la matriz de embeding mayor para que la perturbación no afecte mucho.

Como veremos más adelante en los mecanismos de atención, el valor absoluto de las matrices no afecta a la hora de calcular la atención

Por último se suma el embedding resultante con la matriz de positional encoding. No se suma la matriz de positional encoding entera, sino que se le suma la metriz de positional encoding con tantas filas como tenga `x`.

Como hemos dicho para el entrenamiento se necesita definir una longitud máxima de las frases, de manera que las frases que tengan menos palabras se rellenarán con palabras vacías o sin significado. Pero eso es en el entrenamiento, si hemos definido una longitud máxima de 10 palabras, pero al encoder le metemos 4 palabras, no tiene sentido sumarle la matriz correspondiente a 10 palabras, sino la de 4 plabras. Por eso solo se le suma la matriz con tantas filas (`[:,:sequence_len]`) como palabras tenga `x`.

Hasta ahora, y para no liar, he estado hablando de cuantas palabras puede tener como máximo las frases o cuantas palabras tiene la frase que se introduce para inferencia. Pero esto en realidad no es así

Como hemos explicado, las posibles palabras del vocabulario que tengamos se convierten a `token`s que es la descomposición de estas palabras en otras más sencillas, y a eso le llamamos `token`s. Por tanto la longitud máxima de cada frase en realidad es la cantidad máxima de `token`s que puede tener una frase.

Si quieres puede volver a leer todo el texto anterior y cambia palabras de frases por `token`s. A partir de ahora intentaré referirme a `token`s en vez de palabras, pero es posible que alguna vez se me escape el término palabra.

Vamos a inicializar un objeto de la clase `PositionalEncoding` y ver su matriz de positional encoding

In [24]:
embedding_dim = 4
max_sequence_len = 10
positional_encoding = PositionalEncoding(max_sequence_len=max_sequence_len, embedding_model_dim=embedding_dim)
positional_encoding.positional_encoding.squeeze(0), positional_encoding.positional_encoding.squeeze(0).shape

(tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  9.9995e-01,  1.0000e-04,  1.0000e+00],
         [ 9.0930e-01,  9.9980e-01,  2.0000e-04,  1.0000e+00],
         [ 1.4112e-01,  9.9955e-01,  3.0000e-04,  1.0000e+00],
         [-7.5680e-01,  9.9920e-01,  4.0000e-04,  1.0000e+00],
         [-9.5892e-01,  9.9875e-01,  5.0000e-04,  1.0000e+00],
         [-2.7942e-01,  9.9820e-01,  6.0000e-04,  1.0000e+00],
         [ 6.5699e-01,  9.9755e-01,  7.0000e-04,  1.0000e+00],
         [ 9.8936e-01,  9.9680e-01,  8.0000e-04,  1.0000e+00],
         [ 4.1212e-01,  9.9595e-01,  9.0000e-04,  1.0000e+00]]),
 torch.Size([10, 4]))

Como vemos hemos definido que la máxima longitud de las frases sea 10, la matriz tiene 10 filas, y como hemos definido la longitud el word embedding en 4, la matriz tiene 4 columnas

Si creamos una frase de 10 o menos tokens y obtenemos sus embeddings, lo que hará el transformer será sumarle la matriz de positional encoding, por lo que obtendremos una nueva matriz que corresponderá a un nuevo embedding

## Ejemplos

Vamos a ver unos pocos ejemplos y luego contamos más cosas

Como hemos visto la capa de embedding nos da unos vectores de los tokens

In [25]:
import torch
import torch.nn as nn

class Embedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        """
        Args:
            vocab_size: size of vocabulary
            embed_dim: dimension of embeddings
        """
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            out: embedding vector
        """
        return self.embedding(x)

In [26]:
vocab_size = 100
embedding = Embedding(vocab_size=vocab_size, embedding_dim=embedding_dim)
input = torch.LongTensor([[0, 1, 80, 32, 6]])
word_embeding = embedding(input)
word_embeding

tensor([[[ 0.6896, -0.6904,  0.1987, -1.4398],
         [-1.4949, -0.0955,  1.1580,  0.7795],
         [-1.0794,  1.0133,  0.3143, -0.0952],
         [ 0.9992, -0.8670, -2.1891, -1.0610],
         [-1.1142,  1.4546,  0.8917,  0.9920]]], grad_fn=<EmbeddingBackward0>)

Hemos obtenido los embeddings de los tokens 0, 1, 80, 32 y 6

Como vemos, tenemos cinco vectores que no nos dicen nada, no sabemos si puestos en ese orden corresponden a cuatro palabras que crean una frase con lógica o son cuatro palabras, una detrás de la otra, sin ningún sentido.

Vamos ahora a obtener los embeddings después de pasar por la capa de positional encoding

In [35]:
word_encoding = positional_encoding(word_embeding)
print(f"word_embeding:\n{word_embeding.squeeze(0).detach().numpy()}")
print(f"\nword_encoding:\n{word_encoding.squeeze(0).detach().numpy()}")   # Se añade detach() para que no se muestre el gradiente y se convierte a numpy para que se muestre como array

word_embeding:
[[ 0.68958026 -0.6903649   0.19872816 -1.4398044 ]
 [-1.4948796  -0.09545432  1.15803     0.7795482 ]
 [-1.0793551   1.0132569   0.3142546  -0.09520651]
 [ 0.9991969  -0.8670151  -2.1890657  -1.0609891 ]
 [-1.1142241   1.4546158   0.89166063  0.99203336]]

word_encoding:
[[ 1.3791605  -0.3807298   0.39745632 -1.8796089 ]
 [-2.1482882   0.8090414   2.31616     2.5590963 ]
 [-1.2494128   3.0263138   0.6287092   0.809587  ]
 [ 2.1395137  -0.7344802  -4.3778315  -1.1219783 ]
 [-2.9852507   3.9084318   1.7837212   2.9840667 ]]


Se puede ver que se ha modificado la matriz, veamos en cuanto

In [39]:
print((word_encoding - word_embeding).squeeze(0).detach().numpy())

[[ 0.68958026  0.3096351   0.19872816 -0.43980443]
 [-0.65340865  0.9044957   1.1581299   1.7795482 ]
 [-0.17005765  2.0130568   0.3144546   0.9047935 ]
 [ 1.1403168   0.13253492 -2.1887658  -0.06098914]
 [-1.8710266   2.453816    0.8920606   1.9920334 ]]


Se puede ver que hay algún valor mayor que 1 o menor que -1, pero eso es porque internamente el embedding se ha multiplicado por la raiz cuadrada de la dimensión del word emebedding, si vemos la diferencia de la matriz de embedding después de esa operación y después de sumarle la matriz de positional encoding ya no pasará esto

In [40]:
print((word_encoding - (word_embeding * math.sqrt(embedding_dim))).squeeze(0).detach().numpy())

[[ 0.0000000e+00  1.0000000e+00  0.0000000e+00  1.0000000e+00]
 [ 8.4147096e-01  9.9995005e-01  9.9897385e-05  9.9999988e-01]
 [ 9.0929747e-01  9.9979997e-01  1.9997358e-04  1.0000000e+00]
 [ 1.4111996e-01  9.9955004e-01  2.9993057e-04  1.0000000e+00]
 [-7.5680256e-01  9.9920011e-01  3.9994717e-04  1.0000000e+00]]


Ya no hay ningún valor mayor que 1 o menor que -1

Hemos visto que los valores de la matriz de positional encoding dependen de la posición del token (`pos`), del índice del vector de embedding (`i`) y de la dimensión del word embedding (`d_model`), así que vamos a ver dos matrices de positional encoding don diferentes valores

In [50]:
max_sequence_len = 10
embedding_model_dim = 4
positional_encoding = PositionalEncoding(max_sequence_len=max_sequence_len, embedding_model_dim=embedding_model_dim)
print(f"matriz de positional encoding con max len {max_sequence_len} y embedding dim {embedding_model_dim}:\n {positional_encoding.positional_encoding.squeeze(0).detach().numpy()}")

max_sequence_len = 5
embedding_model_dim = 4
positional_encoding = PositionalEncoding(max_sequence_len=max_sequence_len, embedding_model_dim=embedding_model_dim)
print(f"\nmatriz de positional encoding con max len {max_sequence_len} y embedding dim {embedding_model_dim}:\n {positional_encoding.positional_encoding.squeeze(0).detach().numpy()}")

matriz de positional encoding con max len 10 y embedding dim 4:
 [[ 0.0000000e+00  1.0000000e+00  0.0000000e+00  1.0000000e+00]
 [ 8.4147096e-01  9.9994999e-01  9.9999997e-05  1.0000000e+00]
 [ 9.0929741e-01  9.9980003e-01  1.9999999e-04  1.0000000e+00]
 [ 1.4112000e-01  9.9955004e-01  2.9999999e-04  1.0000000e+00]
 [-7.5680250e-01  9.9920011e-01  3.9999999e-04  1.0000000e+00]
 [-9.5892429e-01  9.9875027e-01  4.9999997e-04  1.0000000e+00]
 [-2.7941549e-01  9.9820054e-01  5.9999997e-04  1.0000000e+00]
 [ 6.5698659e-01  9.9755102e-01  6.9999992e-04  1.0000000e+00]
 [ 9.8935825e-01  9.9680173e-01  7.9999992e-04  1.0000000e+00]
 [ 4.1211849e-01  9.9595273e-01  8.9999987e-04  1.0000000e+00]]

matriz de positional encoding con max len 5 y embedding dim 4:
 [[ 0.0000000e+00  1.0000000e+00  0.0000000e+00  1.0000000e+00]
 [ 8.4147096e-01  9.9994999e-01  9.9999997e-05  1.0000000e+00]
 [ 9.0929741e-01  9.9980003e-01  1.9999999e-04  1.0000000e+00]
 [ 1.4112000e-01  9.9955004e-01  2.9999999e-04  1.

## Cambio de la matriz de positional encoding

Cuando cambia solo el támaño máximo de frase los primeros vectores se mantienen iguales, ya que tanto `pos` como `i` como `d_model` en ellos son iguales

In [55]:
max_sequence_len = 5
embedding_model_dim = 4
positional_encoding = PositionalEncoding(max_sequence_len=max_sequence_len, embedding_model_dim=embedding_model_dim)
print(f"matriz de positional encoding con max len {max_sequence_len} y embedding dim {embedding_model_dim}:\n {positional_encoding.positional_encoding.squeeze(0).detach().numpy()}")

max_sequence_len = 5
embedding_model_dim = 6
positional_encoding = PositionalEncoding(max_sequence_len=max_sequence_len, embedding_model_dim=embedding_model_dim)
print(f"\nmatriz de positional encoding con max len {max_sequence_len} y embedding dim {embedding_model_dim}:\n {positional_encoding.positional_encoding.squeeze(0).detach().numpy()}")

matriz de positional encoding con max len 5 y embedding dim 4:
 [[ 0.0000000e+00  1.0000000e+00  0.0000000e+00  1.0000000e+00]
 [ 8.4147096e-01  9.9994999e-01  9.9999997e-05  1.0000000e+00]
 [ 9.0929741e-01  9.9980003e-01  1.9999999e-04  1.0000000e+00]
 [ 1.4112000e-01  9.9955004e-01  2.9999999e-04  1.0000000e+00]
 [-7.5680250e-01  9.9920011e-01  3.9999999e-04  1.0000000e+00]]

matriz de positional encoding con max len 5 y embedding dim 6:
 [[ 0.0000000e+00  1.0000000e+00  0.0000000e+00  1.0000000e+00
   0.0000000e+00  1.0000000e+00]
 [ 8.4147096e-01  9.9892300e-01  2.1544329e-03  1.0000000e+00
   4.6415889e-06  1.0000000e+00]
 [ 9.0929741e-01  9.9569422e-01  4.3088561e-03  1.0000000e+00
   9.2831779e-06  1.0000000e+00]
 [ 1.4112000e-01  9.9032068e-01  6.4632590e-03  9.9999994e-01
   1.3924767e-05  1.0000000e+00]
 [-7.5680250e-01  9.8281395e-01  8.6176321e-03  9.9999994e-01
   1.8566356e-05  1.0000000e+00]]


Pero cuando cambia el valor de la dimensión del word embedding, solo se menatiene igual el primer vector ya que `d_model` es distinto. El primer vector es igual en ambos casos porque `i = 0`

## Cambio de un token en función de su posición

Vamos a crear 5 frases distintas, en las que una palabra se va a mover por todas las posibles posiciones, vamos a ver como su embedding va a ser distinto después de pasar por la capa de positional encoding

In [59]:
vocab_size = 100
embedding_dim = 4
max_sequence_len = 5
token = 21

input1 = torch.LongTensor([[token, 1, 80, 32, 6]])
input2 = torch.LongTensor([[0, token, 80, 32, 6]])
input3 = torch.LongTensor([[0, 1, token, 32, 6]])
input4 = torch.LongTensor([[0, 1, 80, token, 6]])
input5 = torch.LongTensor([[0, 1, 80, 32, token]])

embedding = Embedding(vocab_size=vocab_size, embedding_dim=embedding_dim)
positional_encoding = PositionalEncoding(max_sequence_len=max_sequence_len, embedding_model_dim=embedding_dim)

word_embeding1 = embedding(input1)
word_embeding2 = embedding(input2)
word_embeding3 = embedding(input3)
word_embeding4 = embedding(input4)
word_embeding5 = embedding(input5)

word_encoding1 = positional_encoding(word_embeding1)
word_encoding2 = positional_encoding(word_embeding2)
word_encoding3 = positional_encoding(word_embeding3)
word_encoding4 = positional_encoding(word_embeding4)
word_encoding5 = positional_encoding(word_embeding5)

print(f"Embedding de la palabra {token} en la posición 0: {word_embeding1.squeeze(0).detach().numpy()[0]}")
print(f"Embedding de la palabra {token} en la posición 1: {word_embeding2.squeeze(0).detach().numpy()[1]}")
print(f"Embedding de la palabra {token} en la posición 2: {word_embeding3.squeeze(0).detach().numpy()[2]}")
print(f"Embedding de la palabra {token} en la posición 3: {word_embeding4.squeeze(0).detach().numpy()[3]}")
print(f"Embedding de la palabra {token} en la posición 4: {word_embeding5.squeeze(0).detach().numpy()[4]}")
print()
print(f"Embedding de la palabra {token} en la posición 0 tras positional encoding: {word_encoding1.squeeze(0).detach().numpy()[0]}")
print(f"Embedding de la palabra {token} en la posición 1 tras positional encoding: {word_encoding2.squeeze(0).detach().numpy()[1]}")
print(f"Embedding de la palabra {token} en la posición 2 tras positional encoding: {word_encoding3.squeeze(0).detach().numpy()[2]}")
print(f"Embedding de la palabra {token} en la posición 3 tras positional encoding: {word_encoding4.squeeze(0).detach().numpy()[3]}")
print(f"Embedding de la palabra {token} en la posición 4 tras positional encoding: {word_encoding5.squeeze(0).detach().numpy()[4]}")

Embedding de la palabra 21 en la posición 0: [ 0.36092606  1.0631591   1.4397429  -0.8110142 ]
Embedding de la palabra 21 en la posición 1: [ 0.36092606  1.0631591   1.4397429  -0.8110142 ]
Embedding de la palabra 21 en la posición 2: [ 0.36092606  1.0631591   1.4397429  -0.8110142 ]
Embedding de la palabra 21 en la posición 3: [ 0.36092606  1.0631591   1.4397429  -0.8110142 ]
Embedding de la palabra 21 en la posición 4: [ 0.36092606  1.0631591   1.4397429  -0.8110142 ]

Embedding de la palabra 21 en la posición 0 tras positional encoding: [ 0.7218521   3.1263182   2.8794858  -0.62202835]
Embedding de la palabra 21 en la posición 1 tras positional encoding: [ 1.563323    3.1262681   2.8795857  -0.62202835]
Embedding de la palabra 21 en la posición 2 tras positional encoding: [ 1.6311495   3.1261182   2.8796859  -0.62202835]
Embedding de la palabra 21 en la posición 3 tras positional encoding: [ 0.86297214  3.1258683   2.8797858  -0.62202835]
Embedding de la palabra 21 en la posición 4 

Como se puee ver el vector de embedding del token 21 es el mismo siempre, pero después de pasar por la capa de positional encoding ya no es igual

## ¿Cómo sabe el transformer la posición de una palabra tras pasar por la capa de positional encoding?

Realmente el transformer, después de la capa de positional encoding, no sabe la posición de cada token. Lo que hace es que en la capa de atención, los tokens que están más cercanos unos de las otros tienen una puntuación mayor que si esos mismos tokens están alejados. Veamos un ejemplo

Vamos a contruir una frase con el mismo token, vamos a obtener sus embeddings, vamos a pasarlo por la capa de positional encoding y vamos a ver la similitud de ese token consigo mismo pero en diferentes posiciones

In [63]:
from torch.nn.functional import cosine_similarity

vocab_size = 100
embedding_dim = 512
max_sequence_len = 5
token = 21

input = torch.LongTensor([[token, token, token, token, token]])

embedding = Embedding(vocab_size=vocab_size, embedding_dim=embedding_dim)
positional_encoding = PositionalEncoding(max_sequence_len=max_sequence_len, embedding_model_dim=embedding_dim)

word_embeding = embedding(input)
word_encoding = positional_encoding(word_embeding)

similarity_token_position0_position0 = cosine_similarity(word_encoding[:,0,:], word_encoding[:,0,:], dim=1)
similarity_token_position0_position1 = cosine_similarity(word_encoding[:,0,:], word_encoding[:,1,:], dim=1)
similarity_token_position0_position2 = cosine_similarity(word_encoding[:,0,:], word_encoding[:,2,:], dim=1)
similarity_token_position0_position3 = cosine_similarity(word_encoding[:,0,:], word_encoding[:,3,:], dim=1)
similarity_token_position0_position4 = cosine_similarity(word_encoding[:,0,:], word_encoding[:,4,:], dim=1)

print(f"Similitud entre el token {token} consigo mismo entre las posiciones 0 y 0 es: {similarity_token_position0_position0.squeeze(0).detach().numpy()*1000}")
print(f"Similitud entre el token {token} consigo mismo entre las posiciones 0 y 1 es: {similarity_token_position0_position1.squeeze(0).detach().numpy()*1000}")
print(f"Similitud entre el token {token} consigo mismo entre las posiciones 0 y 2 es: {similarity_token_position0_position2.squeeze(0).detach().numpy()*1000}")
print(f"Similitud entre el token {token} consigo mismo entre las posiciones 0 y 3 es: {similarity_token_position0_position3.squeeze(0).detach().numpy()*1000}")
print(f"Similitud entre el token {token} consigo mismo entre las posiciones 0 y 4 es: {similarity_token_position0_position4.squeeze(0).detach().numpy()*1000}")

Similitud entre el token 21 consigo mismo entre las posiciones 0 y 0 es: 1000.0001192092896
Similitud entre el token 21 consigo mismo entre las posiciones 0 y 1 es: 999.988317489624
Similitud entre el token 21 consigo mismo entre las posiciones 0 y 2 es: 999.9597668647766
Similitud entre el token 21 consigo mismo entre las posiciones 0 y 3 es: 999.9275207519531
Similitud entre el token 21 consigo mismo entre las posiciones 0 y 4 es: 999.9014735221863


Como se puede ver la similitud entre un mismo token es menor a medida que se separa en una frase. Por lo que gracias al mecanismo de positional encoding, cuando calculemos la relación entre tokens mediante el mecanismo de atención por un lado obtendremos la similitud semantica entre tokens y por otro lado obtendremos cómo de cerca o lejos están en una frase